In [1]:
import os
import numpy as np
import cv2
import torch
from torch.utils.data import Dataset


class CoronarySmallDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx])
        

        image = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
        image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)
        image = cv2.resize(image, (256, 256))
        
        mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
        mask = cv2.cvtColor(mask, cv2.COLOR_RGBA2RGB)
        mask = cv2.resize(mask, (256, 256))
        
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)
        
        return image, mask

In [2]:
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms


transform = transforms.Compose([
    transforms.ToTensor()
])

train_image_dir = 'images_train\input'
train_mask_dir = 'images_train\output'
val_image_dir = 'images_val\input'
val_mask_dir = 'images_val\output'
test_image_dir = 'images_test\input'
test_mask_dir = 'images_test\output'

train_dataset = CoronarySmallDataset(train_image_dir, train_mask_dir, transform=transform)
val_dataset = CoronarySmallDataset(val_image_dir, val_mask_dir, transform=transform)
test_dataset = CoronarySmallDataset(test_image_dir, test_mask_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=12, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=12, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=12, shuffle=False)


In [3]:
from medium_RGB_model import UNet


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet()
model = model.to(device)

In [4]:
import torch.optim as optim
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F


criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=50):
    best_loss = float('inf')
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for images, masks in tqdm(train_loader):
            images = images.to(device)
            masks = masks.to(device)

            # print(images.size())
            # print(images)
            # print(masks.size())
            # print(masks)

            optimizer.zero_grad()
            outputs = model(images)

            # print(outputs.size())
            # print(outputs)
            
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * images.size(0)
        
        train_loss = train_loss / len(train_loader.dataset)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, masks in val_loader:
                images = images.to(device)
                masks = masks.to(device)

                outputs = model(images)
                loss = criterion(outputs, masks)
                
                val_loss += loss.item() * images.size(0)
        
        val_loss = val_loss / len(val_loader.dataset)
        
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
        
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), 'medium_RGB_model.pth')


In [5]:
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=35)

100%|██████████| 47/47 [01:49<00:00,  2.33s/it]


Epoch 1/35, Train Loss: 0.6006, Val Loss: 0.5492


100%|██████████| 47/47 [02:06<00:00,  2.69s/it]


Epoch 2/35, Train Loss: 0.5271, Val Loss: 0.5071


100%|██████████| 47/47 [02:06<00:00,  2.70s/it]


...

Epoch 33/35, Train Loss: 0.1158, Val Loss: 0.1201


100%|██████████| 47/47 [01:55<00:00,  2.45s/it]


Epoch 34/35, Train Loss: 0.1118, Val Loss: 0.1130


100%|██████████| 47/47 [01:55<00:00,  2.46s/it]


Epoch 35/35, Train Loss: 0.1079, Val Loss: 0.1121


In [6]:
import os
import numpy as np
import cv2
import torch
from torchvision import transforms


model.load_state_dict(torch.load('medium_RGB_model.pth'))
model.eval()

def show_image(type, image_name):
    dir = f"images_test\{type}"
    print(dir)
    img = cv2.imread(os.path.join(dir, image_name), cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
    img = cv2.resize(img, (256, 256))
    cv2.imshow(type, img)

def predict(image_name):
    dir = 'images_test\input'
    img = cv2.imread(os.path.join(dir, image_name), cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
    img = cv2.resize(img, (256, 256))
    img = transforms.ToTensor()(img).unsqueeze(0).to(device)

    with torch.no_grad():
        pred = model(img)
        pred = pred.squeeze().cpu().numpy()
        print(pred)
        # pred = (pred > 0.5).astype(np.uint8)
    R, G, B = pred[0], pred[1], pred[2]
    # cv2.imshow('R', R)
    # cv2.imshow('G', G)
    # cv2.imshow('B', B)
    pred_image = np.zeros((256, 256, 3), dtype=np.uint8)
    pred_image[..., 0] = (R * 255).astype(np.uint8)
    pred_image[..., 1] = (G * 255).astype(np.uint8)
    pred_image[..., 2] = (B * 255).astype(np.uint8)
    print(pred_image)
    cv2.imshow('pred', pred_image)
    cv2.waitKey(0)
   

In [10]:
# image_name = "13c2ur549vohc0jat2dvu3xs7q1_18.png"
# image_name = "13c2ur549vohc0jat2wd23me01_25.png"
# image_name = "131aedfhs6pnf1fvtvp49h4bhdjeabmt22_31.png"
image_name = "131aedfhs6pnf1fvtvp49juwu7plj9dv22_40.png"
show_image("input", image_name)
show_image("output", image_name)
predict(image_name)

images_test\input
images_test\output
[[[0.08210416 0.07659057 0.08195937 ... 0.0795963  0.08641205 0.11238441]
  [0.07283241 0.06846111 0.07316695 ... 0.07646154 0.07605398 0.11471844]
  [0.07830185 0.0757615  0.08097625 ... 0.08816575 0.08847605 0.08901044]
  ...
  [0.07874265 0.07522918 0.07968123 ... 0.08675385 0.07669307 0.08534547]
  [0.07539952 0.0688486  0.06573147 ... 0.06784961 0.0684299  0.09235587]
  [0.10136071 0.08428089 0.08912286 ... 0.07137823 0.07984713 0.10630883]]

 [[0.08359675 0.06037613 0.06536017 ... 0.0698146  0.08078489 0.09377851]
  [0.06048901 0.04921501 0.05424129 ... 0.06859186 0.07100367 0.09573403]
  [0.06376133 0.05666051 0.06480587 ... 0.08127629 0.08614729 0.08110905]
  ...
  [0.06378844 0.05253183 0.06383098 ... 0.08244563 0.08341922 0.09178697]
  [0.05795839 0.04639269 0.04949585 ... 0.05628017 0.07413596 0.0816901 ]
  [0.08232568 0.06948467 0.06603094 ... 0.05872029 0.07072691 0.10848798]]

 [[0.10226918 0.07861469 0.08995807 ... 0.09780356 0.113887

In [11]:
model.load_state_dict(torch.load('medium_RGB_model.pth'))

def dice_coefficient(y_true, y_pred):
    smooth = 1.0
    y_true_f = y_true.view(-1)
    y_pred_f = y_pred.view(-1)
    intersection = (y_true_f * y_pred_f).sum()
    return (2. * intersection + smooth) / (y_true_f.sum() + y_pred_f.sum() + smooth)

model.eval()
test_loss = 0.0
dice_scores = []

with torch.no_grad():
    for images, masks in test_loader:
        images = images.to(device)
        masks = masks.to(device)

        outputs = model(images)
        loss = criterion(outputs, masks)
        
        test_loss += loss.item() * images.size(0)
        dice_scores.append(dice_coefficient(masks, outputs))

test_loss = test_loss / len(test_loader.dataset)
mean_dice = torch.mean(torch.tensor(dice_scores))

print(f'Test Loss: {test_loss:.4f}')
print(f'Mean Dice Coefficient: {mean_dice:.4f}')


Test Loss: 0.5773
Mean Dice Coefficient: 0.6904


In [ ]:
from skimage import morphology

def post_process(prediction):
    prediction_np = prediction.cpu().numpy()
    processed = morphology.remove_small_objects(prediction_np > 0.5, min_size=100)
    processed = morphology.remove_small_holes(processed, area_threshold=100)
    return torch.tensor(processed, device=device)

post_processed_predictions = [post_process(pred) for pred in outputs]


In [ ]:
from flask import Flask, request, jsonify
import io

app = Flask(__name__)
model.load_state_dict(torch.load('model.pth'))
model.eval()

@app.route('/predict', methods=['POST'])
def predict():
    file = request.files['image']
    img_bytes = file.read()
    img = cv2.imdecode(np.frombuffer(img_bytes, np.uint8), cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    img = cv2.resize(img, (512, 512))
    img = transforms.ToTensor()(img).unsqueeze(0).to(device)

    with torch.no_grad():
        pred = model(img)
        pred = pred.squeeze().cpu().numpy()
        pred = (pred > 0.5).astype(np.uint8)
    
    _, buffer = cv2.imencode('.png', pred)
    response = io.BytesIO(buffer)
    response.seek(0)
    return response, 200, {'Content-Type': 'image/png'}

if __name__ == '__main__':
    app.run()
